In [1]:
import pandas as pd

lab_clean = pd.read_csv(r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\testing.csv", sep = "|")

C:\Users\mhuertas\AppData\Local\Temp\ipykernel_7912\3999169792.py:3: DtypeWarning: Columns (1,7,8,9,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  lab_clean = pd.read_csv(r"U:\Estudis\B52_CORDELIA\Dades\PADRIS\testing.csv", sep = "|")


In [2]:
# -----------------------------------------
# ----- Step 1: Results with reportable limits -> flag implausible results

lab_clean.head()

,codi_p,peticio_id,any,data,codi_prova,prova,resultat,unitat_mesura,ref_min,ref_max,clean_result,clean_unit,comentari,comentari_unitat,num_type
0,10254,81705546520,2010,17/08/2010,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,0,K/mcL,NaN,NaN,0,10^3/µl,NaN,done,n1
1,7314,80100100846,2011,01/08/2011,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,0,K/mcL,NaN,NaN,0,10^3/µl,NaN,done,n1
2,8871,31774972,2008,17/03/2008,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,0,K/mcL,NaN,NaN,0,10^3/µl,NaN,done,n1
3,3241,51354073,2008,13/05/2008,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,0,K/mcL,NaN,NaN,0,10^3/µl,NaN,done,n1
4,11405,42005701677,2010,20/04/2010,H11740,BASÒFILS (HEMOGRAMA); C.NOM. «PART DE CONJUNT»,0,K/mcL,NaN,NaN,0,10^3/µl,NaN,done,n1
